In [3]:
import pandas as pd
import numpy as np

In [4]:

# Connect to the database
import pymysql.cursors
import json

# We will use these lists to generate a table later. 

sessionIds = []
loginIds = []

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_mirror")

try:
    with connection.cursor() as cursor:
        # This query select rows from log_trailer_actions where the action is either TrailerModalLaunched or Recommendations Refreshed
        # These two actions indicate that a recommendation list is generated. 
        
        sql = """
        SELECT user_rating_pairs.userId, user_rating_pairs.movieId, user_rating_pairs.rating, user_rating_pairs.tstamp, movie_genre_pairs.movieId, movie_genre_pairs.genreId FROM user_rating_pairs
        LEFT JOIN movie_genre_pairs
        ON movie_genre_pairs.movieId = user_rating_pairs.movieId

        """ 
        cursor.execute(sql)
        result = cursor.fetchall()
            
finally: 
    connection.close()


In [5]:
movie_rating_genre = pd.DataFrame(list(result), columns=['userId','movieId','rating','tstamp','movieId','genreId'])

In [6]:
movie_rating_genre.head()

,userId,movieId,rating,tstamp,movieId,genreId
0,1,5,3.0,1997-10-17 06:48:20,5.0,5.0
1,1,10,4.0,1997-10-17 05:00:17,10.0,1.0
2,1,10,4.0,1997-10-17 05:00:17,10.0,2.0
3,1,10,4.0,1997-10-17 05:00:17,10.0,16.0
4,1,13,4.0,1997-10-17 06:37:13,13.0,2.0


In [11]:
movie_rating_genre_agg = movie_rating_genre.groupby(["userId","genreId"]).mean().reset_index()



In [12]:
movie_rating_genre_agg.head()

,userId,genreId,movieId,rating,movieId
0,1,1.0,926.821429,3.660714,926.821429
1,1,2.0,927.260000,3.740000,927.260000
2,1,3.0,812.055556,4.388889,812.055556
3,1,4.0,881.083333,3.805556,881.083333
4,1,5.0,897.600000,3.700000,897.600000


In [13]:
movie_rating_genre_agg.to_csv("../movie_rating_genre_agg.csv")

